In [1]:
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/Amazon Fine Food Review/Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
df['Sentiment'] = df["Score"].apply(lambda x: 1 if x > 3 else (0 if x == 3 else -1))

In [6]:
df = df[df.HelpfulnessNumerator <= df.HelpfulnessDenominator]

In [7]:
filtered_df = df.drop_duplicates(subset = {'UserId','ProfileName','Time'} ,keep = 'first', inplace = False)

In [8]:
final = filtered_df.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)

In [9]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1
150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1
451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1
230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1
451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1


In [10]:
final['Sentiment'].value_counts()

 1    273904
-1     52692
 0     26188
Name: Sentiment, dtype: int64

In [11]:
def cleanhtml(sent):# function for cleaning html tags
    cleanr = re.compile('<.*?>')
    cleaned = re.sub(cleanr,' ',sent)
    return cleaned

In [12]:
def cleanpunc(sent):# function for cleaning punctuations
    clean = re.sub(r'[?|!|$|#|\'|"|:]',r'',sent)
    clean = re.sub(r'[,|(|)|.|\|/]',r' ',clean)
    return clean

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
stop = set(stopwords.words("english"))

In [15]:
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer("english")
st.stem('burned')

'burn'

In [16]:
i=0
all_positive_reviews = []
all_negative_reviews = []
all_neutral_reviews = []

final_string = []
stem_data = " "
for p in final['Text'].values:
    filtered_sens = []#filtered word
    p = cleanhtml(p)
    for w in p.split():
       # print(w)
        punc = cleanpunc(w)
        for s in punc.split():
            #print(w)
            if (s.isalpha()) & (len(s)>2):
                if s.lower() not in stop:
                    stem_data = (st.stem(s.lower())).encode('utf8')
                    filtered_sens.append(stem_data)
                    if (final['Sentiment'].values)[i] == 1:
                        all_positive_reviews.append(stem_data)
                    if (final['Sentiment'].values)[i] == -1:
                        all_negative_reviews.append(stem_data)
                    if (final['Sentiment'].values)[i] == 0:
                        all_negative_reviews.append(stem_data)    
                else:
                    continue
            else:
                continue
    #print(filtered_sens)
    str1 = b" ".join(filtered_sens)
    #print(str1)
    final_string.append(str1)
    i+=1

In [17]:
final['CleanedText'] = final_string
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment,CleanedText
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,1,b'witti littl book make son laugh loud recit c...
150500,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,1,b'rememb see show air televis year ago child s...
451855,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,1,b'beetlejuic well written movi everyth excel a...
230284,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",1,b'twist rumplestiskin captur film star michael...
451854,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,5,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,1,b'beetlejuic excel funni movi keaton hilari wa...


In [18]:
#final.to_csv('/content/drive/My Drive/Amazon Fine Food Review/Final_Vivek.csv')

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
final = final.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)
X = final['CleanedText'].values
y = final['Sentiment'].values

In [22]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,stratify = y)

In [23]:
# Logistic Regression on Avg W2Vec

In [25]:
list_of_sent_train = []
for i in X_train:
    sent = []
    for word in i.split():
      
        sent.append(word.decode('utf-8'))
    list_of_sent_train.append(sent)

In [26]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(list_of_sent_train,min_count = 5,size = 50,workers = 4)
sent_vectors_train = []
for sent in list_of_sent_train:
    sent_vec = np.zeros(50)
    cnt_word = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_word += 1
        except:
            pass
    sent_vec /= cnt_word
    sent_vectors_train.append(sent_vec)
print(len(sent_vectors_train))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


246948


In [27]:
list_of_sent_test = []
for i in X_test:
    sent = []
    for word in i.split():
        sent.append(word.decode('utf-8'))
    list_of_sent_test.append(sent)

In [28]:
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
w2v_model = Word2Vec(list_of_sent_test,min_count = 5,size = 50,workers = 4)
sent_vectors_test = []
for sent in list_of_sent_test:
    sent_vec = np.zeros(50)
    cnt_word = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_word += 1
        except:
            pass
    sent_vec /= cnt_word
    sent_vectors_test.append(sent_vec)
print(len(sent_vectors_test))

105836


In [32]:
np.where(np.isnan(sent_vectors_train))

(array([73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162,
        73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162,
        73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162,
        73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162,
        73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162, 73162,
        73162, 73162, 73162, 73162, 73162]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]))

In [37]:
sent_vectors_train[73162]

array([ 0.2705984 ,  0.95360182, -0.66975516, -0.32120155, -0.06392572,
        0.3300283 ,  1.74153457,  0.11497893, -0.30618289, -1.16858152,
       -1.7284313 ,  0.34843092, -0.67558522,  1.13121448, -0.2706077 ,
        0.31483683,  0.66383871, -0.34787408, -0.28628433,  0.17810926,
        0.59972417, -0.60697936, -0.09201705, -0.68831644, -0.9891998 ,
       -0.70775114,  0.90781392, -0.51154707,  0.0967647 ,  0.99367385,
        1.14538652,  0.26399582,  0.95016924, -0.2099863 ,  0.59407045,
        0.31707397,  0.72085396, -0.51385252,  0.05392276, -0.49425036,
       -0.46875679,  0.73838576, -0.29498344, -0.32551856,  0.44975446,
        0.07800849,  0.53034404, -0.91083526, -0.34082449,  0.11125175])

In [36]:
#sent_vectors_train = np.delete(sent_vectors_train, (73162), axis=0)

In [39]:
#y_train = np.delete(y_train, (73162), axis=0)

In [48]:
#y_test = np.delete(y_test, (80293), axis=0)

In [46]:
#sent_vectors_test = np.delete(sent_vectors_test, (80293), axis=0)

In [47]:
np.where(np.isnan(sent_vectors_test))

(array([], dtype=int64), array([], dtype=int64))

In [49]:
from sklearn.preprocessing import StandardScaler
sc =  StandardScaler()
w2v_train = sc.fit_transform(sent_vectors_train)
w2v_test = sc.transform(sent_vectors_test)
param = [{'C':[10**i for i in range(-3,4)]}]
w2v_model = GridSearchCV(LogisticRegression(class_weight = 'balanced',penalty = 'l2'),param,scoring = 'accuracy',cv = 10,n_jobs = -1,)
w2v_model.fit(w2v_train,y_train)
print(w2v_model.best_estimator_)
pred = w2v_model.predict(w2v_test)
acc = accuracy_score(y_test,pred)
print('accuracy is',acc*100)

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
accuracy is 69.25628112214527


In [57]:
#Logistic regression on tfidf

In [59]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
final = final.sort_values('Time',axis= 0,inplace = False , na_position = 'last',ascending = True)
X = final['CleanedText'].values
y = final['Sentiment'].values

In [61]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,stratify = y)

In [65]:
tfidf_vect = TfidfVectorizer()
tfidf_train = tfidf_vect.fit_transform (X_train)
tfidf_test = tfidf_vect.transform(X_test)


In [66]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
X_tr = sc.fit_transform(tfidf_train)
X_te = sc.transform(tfidf_test)
param = [{'C':[10**i for i in range(-3,4)]}]
gd = GridSearchCV(LogisticRegression(class_weight = 'balanced',penalty = 'l2'),param,cv = 10, scoring = 'accuracy',n_jobs = -1)
gd.fit(X_tr,y_train)
print(gd.best_estimator_)
pred = gd.predict(X_te)
print('Accuracy is ',accuracy_score(y_test,pred)*100)

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy is  75.06614006576213


In [67]:
# Logistic Regression on Tfidf W2Vec

In [68]:
tfidf_vect = TfidfVectorizer()
tf_idf_feat = tfidf_vect.get_feature_names()
tfidf_sent_vec_train = []
row = 0
for sent in list_of_sent_train:
    sent_vec = np.zeros(50)
    weight_sum = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            tfidf = tfidf_train[row,tf_idf_feat.index(word)]
            sent_vec += (vec*tfidf)
            weight_sum += tfidf
        except:
            pass
    sent_vec/= weight_sum
    tfidf_sent_vec_train.append(sent_vec)
    row += 1

NotFittedError: ignored

In [ ]:
tf_idf_feat = tfidf_vect.get_feature_names()
tfidf_sent_vec_test = []
row = 0
for sent in list_of_sent_test:
    sent_vec = np.zeros(50)
    weight_sum = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            tfidf = tfidf_test[row,tf_idf_feat.index(word)]
            sent_vec += (vec*tfidf)
            weight_sum += tfidf
        except:
            pass
    sent_vec/= weight_sum
    tfidf_sent_vec_test.append(sent_vec)
    row += 1

In [ ]:
np.where(np.isnan(tfidf_sent_vec_train))

In [ ]:
#del tfidf_sent_vec_train[10706]

In [ ]:
#y_train = np.delete(y_train,10706)

In [ ]:
sc =  StandardScaler()
tfidf_w2v_train = sc.fit_transform(tfidf_sent_vec_train)
tfidf_w2v_test = sc.transform(tfidf_sent_vec_test)
param = [{'C':[10**i for i in range(-3,4)]}]
wv_model = GridSearchCV(LogisticRegression(class_weight = 'balanced',penalty = 'l1'),param,scoring = 'accuracy',cv = 10,n_jobs = -1,)
wv_model.fit(tfidf_w2v_train,y_train)
print(wv_model.best_estimator_)
pred = wv_model.predict(tfidf_w2v_test)
acc = accuracy_score(y_test,pred)
print('accuracy is',acc*100)